In [15]:
import torch as t
import math
import numpy as np

FRAME_TIME = 0.1  # time interval
GRAVITY_ACCEL = 0.12  # gravity constant
BOOST_ACCEL = 0.18  # thrust constant
ROTATION_ACCEL = np.radians(20)  # rotation constant

action = t.tensor([0.5, 1.])
state = t.tensor([0.7854, 0., 0., 0., 0., 0.])

delta_state_ori = ROTATION_ACCEL* FRAME_TIME*t.tensor([0., 1, 0., 0., 0., 0.])

sin = t.matmul(t.tensor([[0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [-1., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.]]),t.sin(state))
cos = t.matmul(t.tensor([[0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [0., 0., 0., 0., 0., 0.],
                         [1., 0., 0., 0., 0., 0.]]),t.cos(state))
print(sin)
print(cos)
cos_sin = cos+sin
delta_state_pos = BOOST_ACCEL* FRAME_TIME*t.tensor([0., 0., 0., 1., 0., 1.])*cos_sin # x and y components of acceleration

delta_state_combine = delta_state_ori + delta_state_pos

action_mat = t.tensor([[0., 0.],
                 [2., 0.],
                 [0., 0.],
                 [0., 1.],
                 [0., 0.],
                 [0., 1.]])
a_a = t.matmul(action_mat, action) + t.tensor([0., -1., 0., 0., 0., 0.]) # get ori and pos accelerations
print(a_a)

delta_state = delta_state_combine*a_a

state_gravity = GRAVITY_ACCEL * FRAME_TIME*t.tensor([0., 0., 0., 0., 0., -1.])
state = state + delta_state + state_gravity

step_mat = t.tensor([[1., FRAME_TIME, 0., 0., 0., 0.],
                     [0., 1., 0., 0., 0., 0.],
                     [0., 0., 1., FRAME_TIME, 0., 0.],
                     [0., 0., 0., 1., 0., 0.],
                     [0., 0., 0., 0., 1., FRAME_TIME,],
                     [0., 0., 0., 0., 0., 1.]])
state = t.matmul(step_mat, state)

print(state)


tensor([ 0.0000,  0.0000,  0.0000, -0.7071,  0.0000,  0.0000])
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7071])
tensor([0., 0., 0., 1., 0., 1.])
tensor([ 7.8540e-01,  0.0000e+00, -1.2728e-03, -1.2728e-02,  7.2790e-05,
         7.2790e-04])


In [74]:
(state[4]*(1 / (1 + math.exp(4*(abs(state[4])-0.25)))))**2

2

In [77]:
a = [7., 6., 5., 4., 3., 2.]
b = [7., 0., 9., 4., 6., 1.]
c = (a,b)
print(c[1])


[7.0, 0.0, 9.0, 4.0, 6.0, 1.0]
